# Лабораторная работа 3. Модель однофазной одноканальной замкнутой системы обслуживания
## Задание 1

In [ ]:
variant <- 5
set.seed(variant)

k <- sample(c(4:9), 1)

pp1 <- runif(4)
pp2 <- runif(3)
pp3 <- runif(2)

p1 <- pp1 / sum(pp1)
p2 <- c(c(0), pp2 / sum(pp2))
p3 <- c(c(0, 0), pp3 / sum(pp3))
p4 <- c(0, 0, 0, 1)

P <- data.frame()

P <- rbind(P, p1, p2, p3, p4)

rownames(P) <- c("p1", "p2", "p3", "p4")
colnames(P) <- c("", "", "", "")

View(P)

print(paste("k =", as.character(k)))

### Граф состояний
С точностью до второго знака после запятой
![graph](./TransitionMatrix_graph--dot.png)

### Численно
Симулируем проход по матрице переходных вероятностей для трех сценариев:
1. k - 2 осмотров,
2. k - 1 осмотров,
3. k осмотров.

Выполнив эту операцию N раз для каждого сценария, получим три вектора,
содержащие N состояний. Это состояния, в которых оставалась модель после
симуляции:
$$
  Scenario_i = \{State_1 ... State_N\}, i \in \{1, 2, 3\}, \\
  State \in \{S1, S2, S3, S4\}
$$

In [ ]:
# Получаем значение, соответствующее отрезку.
# v - случайная величина. Если нужно сгенерировать распределение по `p`, нужно
# подать runif(1)
# X - величины, соответствующие отрезкам.
# За отрезки отвечает параметр p, хранящий последовательность длин.
get_value_in_range <- function(v, X, p) {
    border <- 0
    for (i in seq_along(p)) {
        border <- border + p[i]
        if (v < border) {
            return(X[i])
        }
    }
}

In [ ]:
make_step <- function(current_state) {
    possible_states <- P[current_state, ]

    return(get_value_in_range(runif(1), 1:4, possible_states))
}

In [ ]:
walk <- function(starting_state, times) {
    if (times == 0) {
        return(starting_state)
    }

    next_state <- make_step(starting_state)

    return(walk(next_state, times - 1))
}

In [ ]:
N <- 10000
test_walk <- function(first_state, k) {
    results <- c()

    for (i in 1:N) {
        results <- append(results, walk(first_state, k))
    }

    return(results)
}

In [ ]:
first_state <- 1 # Выбрали произвольное.

In [ ]:
Scenario1 <- test_walk(first_state, k - 2)

In [ ]:
Scenario2 <- test_walk(first_state, k - 1)

In [ ]:
Scenario3 <- test_walk(first_state, k)

В полученных сценариях посчитаем вероятности получения каждого состояния.

In [ ]:
get_probability <- function(States, state) {
    number_of_states <- length(States[States == state])
    number_of_all_states <- length(States)

    return(number_of_states / number_of_all_states)
}

In [ ]:
get_probabilities_to_stay <- function(States) {
    return(
        unlist(
            lapply(seq_along(P), function(state) get_probability(States, state))
        )
    )
}

In [ ]:
Scenario1Propabilities <- get_probabilities_to_stay(Scenario1)

In [ ]:
Scenario2Propabilities <- get_probabilities_to_stay(Scenario2)

In [ ]:
Scenario3Propabilities <- get_probabilities_to_stay(Scenario3)

In [ ]:
results <- data.frame(
    Scenario1Propabilities,
    Scenario2Propabilities,
    Scenario3Propabilities
)
colnames(results) <- c("k - 2", "k - 1", "k")
rownames(results) <- rownames(P)

results

### Теоретически

In [ ]:
initial_state_probabilities <- c(1, 0, 0, 0)
transition_matrix <- data.matrix(P)
transition_matrix

In [ ]:
if (!require("matrixcalc")) {
    install.packages("matrixcalc")
}

initial_state_probabilities %*% matrix.power(transition_matrix, k)

Как видно, теоретически вычисленная матрица с некоторой точностью совпадает
со значениями, полученными теоретически. При увеличении количества
экспериментов $N$ точность только увеличивается.

# Задание 2

In [76]:
variant <- 5
set.seed(variant)
k <- sample(c(10:25), 1)
# Интесивность получалась больше 1, поэтому заменил значение.
# t1 <- sample(c(14:20), 1)
t1 <- 50
t2 <- sample(c(2:5), 1)
View(data.frame(k, t1, t2))

k,t1,t2
<int>,<dbl>,<int>
11,50,4


### Численно

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
она  же обратная вероятность того, что
программа **будет выполнена** сразу же, то есть:

In [77]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [78]:
programmers <- trajectory("programmers' path") %>%
    ## add an intake activity
    seize("server", 1) %>%
    timeout(function() rexp(1, 1 / t2)) %>%
    release("server", 1)

In [79]:
env %>%
    add_resource("server", 1) %>%
    add_generator("programmers", programmers, function() rexp(1, k / t1))
# , function() rnorm(1, 0.1, 2))

simmer environment: SuperDuperSim | now: 0 | next: 0
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programmers | monitored: 1 | n_generated: 0 }

In [80]:
env %>%
    reset() %>%
    run(500)

simmer environment: SuperDuperSim | now: 500 | next: 502.279847724964
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 1(1) | queue status: 2(Inf) }
{ Source: programmers | monitored: 1 | n_generated: 109 }

In [82]:
df <- env %>% get_mon_resources()
df

resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,0.3297179,1,0,1,Inf,1,Inf,1
server,0.5533978,0,0,1,Inf,0,Inf,1
server,2.1575130,1,0,1,Inf,1,Inf,1
server,2.4637538,0,0,1,Inf,0,Inf,1
server,4.9569239,1,0,1,Inf,1,Inf,1
server,8.9349370,1,1,1,Inf,2,Inf,1
server,12.9621081,1,0,1,Inf,1,Inf,1
server,14.5006801,0,0,1,Inf,0,Inf,1
server,14.5860741,1,0,1,Inf,1,Inf,1


In [89]:
queue <- df$queue
income_count <- length(queue)
programs_held <- length(queue[queue != 0])

In [93]:
program_wont_be_executed_immediately <- programs_held / income_count
program_wont_be_executed_immediately

[1] 0.8732394

### Теоретически

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
она  же обратная вероятность того, что
программа **будет выполнена** сразу же, то есть:
$$
1 - P_0 = 1 - (1 - \rho) = \rho = \frac{\lambda}{\mu}
$$
где $\lambda$ - интенсивность, с которой заявки приходят:
$$
\lambda = \frac{k}{t_1} \\
$$

In [ ]:
income_intensity <- k / t1
income_intensity

а $\mu$ - интенсивность обслуживания:
$$
\mu = \frac{1}{t_2}
$$

In [ ]:
process_intensity <- 1 / t2
process_intensity

In [ ]:
program_wont_be_executed_immediately <- income_intensity / process_intensity
program_wont_be_executed_immediately

#### 2. Среднее время до получения пользователем результатов реализации.

Оно же среднее время пребывания заявки в системе по формуле Литтла:
$$
T_{\text{сист}} = \frac{1}{\mu(1 - \rho)}
$$

In [ ]:
time_to_get <- 1 / process_intensity / (1 - program_wont_be_executed_immediately)
time_to_get

#### 3. Среднее количество программ, ожидающих выполнения на сервере.

Она же средняя длина очереди $L_{\text{оч}} = \frac{\rho ^ 2}{1 - \rho}$:

In [ ]:
mean_queue <- program_wont_be_executed_immediately^2 / (1 - program_wont_be_executed_immediately)
mean_queue